In [ ]:
!pip install pymongo
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from datetime import datetime
import time
import json
import gc
import torch
import pandas as pd

In [ ]:
!python -m pip install einops

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Load once (global)
embedding_model = SentenceTransformer(
    "nomic-ai/nomic-embed-text-v1.5",
    trust_remote_code=True
)

def embed_query(query_text):
    prefix = "classification: "
    return embedding_model.encode(
        prefix + query_text,
        normalize_embeddings=True
    ).tolist()

<All keys matched successfully>


In [ ]:
def search_dense(collection, query_text, k=10, index_name="lrcm_dense"):
    embedding = embed_query(query_text)

    """
    Execute semantic vector search
    """
    # Load the same model you used for indexing
    model = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', trust_remote_code=True)

    # Apply the same prefix used during indexing
    query_with_prefix = "search_query: " + query_text
    query_vector = model.encode(query_with_prefix, normalize_embeddings=True).tolist()
    pipeline = [
        {
            "$vectorSearch": {
                "index": "lrcm_dense",
                "path": "production_embedding.embedding",
                "queryVector": embedding,
                "numCandidates": k + 50,  # Cast a wider net
                "limit": k
            }
        },
        {
            "$project": {
                "ticker": 1,
                "name": 1,
                "prod_text":1,
                "score": {"$meta": "vectorSearchScore"}
            }
        }
    ]

    results = list(collection.aggregate(pipeline))
    return pd.DataFrame(results)


In [ ]:
import pymongo

db_username = 'username'
db_password = 'password'
URI = f'url'

client = pymongo.MongoClient(URI)

db = client["project3"]
collection = db["wikipedia_holdings"]
print("Connected to MongoDB.")

Connected to MongoDB.


In [ ]:
themes = [
'Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure',
'Quantum Computing, Next-Gen Computing & Advanced Semiconductors',
'Robotics, Automation, Autonomous Vehicles & Industrial AI Systems',
'AI-Powered Cybersecurity & Network Security Platforms',
'Digital Finance, Payments, Neobanks & Fintech Infrastructure',
'Metaverse, AR/VR, Gaming & Digital Reality Platforms',
'Gene Editing, Synthetic Biology, AI Drug Discovery & Digital Health',
'AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software',
'Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure',
'Nuclear, Renewable Energy, Grid Storage & Power for Digital Infrastructure',
]

In [ ]:
dataframe_list = []
for theme in themes:
  dense_df = search_dense(collection, theme, k=30)
  dataframe_list.append(dense_df)

dataframe_list[0]

<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>


,_id,ticker,name,prod_text,score
0,690c2f0530de78ca9390aa24,COIN,COINBASE GLOBAL INC CLASS A,**Regulatory Resolution:** Successfully resol...,0.862405
1,690c2f0530de78ca9390acc2,CRCL,CIRCLE INTERNET GROUP INC CLASS A,"Circle's revenue model is diversified, built ...",0.849683
2,690c2f0530de78ca9390aa22,MSTR,STRATEGY INC CLASS A,Rebranding from MicroStrategy Incorporated in...,0.849596
3,690c2f0530de78ca9390aa7d,XYZ,BLOCK INC CLASS A,**Strategic Shift to Bitcoin Ecosystem:** XYZ...,0.847160
4,690c2f0530de78ca9390aa3a,DLR,DIGITAL REALTY TRUST REIT INC,Digital Realty is prioritizing sustainable gr...,0.841182
5,690c2f0530de78ca9390a9f4,CME,CME GROUP INC CLASS A,**Dominant Market Position & Growth:** CME Gr...,0.838476
6,690c2f0530de78ca9390abda,AKAM,AKAMAI TECHNOLOGIES INC,**Strategic Cloud Partnership:** Secured a mu...,0.835591
7,690c2f0530de78ca9390ad3f,DJT,TRUMP MEDIA TECHNOLOGY GROUP CORP,"Completed merger with DWAC on March 25, 2024,...",0.831252
8,690c2f0530de78ca9390aa3d,FCX,FREEPORT MCMORAN INC,**Dominant Copper Producer:** The 2007 acquis...,0.830412
9,690c2f0530de78ca9390a9c5,BLK,BLACKROCK INC,**Dominant Market Position & Growth:** BlackR...,0.829773


In [ ]:
import ollama
import json
from pydantic import BaseModel
from typing import List
from typing import Literal

class DetermineTheme(BaseModel):
    ticker: str
    # Literal restricts the string to ONLY these three values
    classification: Literal["core", "secondary", "not_relevant"]
    reasoning: str

thematic_results = []

for theme in themes:
    dense_df = search_dense(collection, theme, k=30)

    for row in dense_df.itertuples():
        try:
            response = ollama.chat(
                model="gemma3n:e2b",
                options={
                    "num_predict": 2500,
                    "temperature": 0.0,
                    "repeat_penalty": 1.2
                },
                format=DetermineTheme.model_json_schema(),
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "You are a senior equity analyst evaluating stock selections for strict alignment with a specific investment theme."
                        )
                    },
                    {
                        "role": "user",
                        "content": f"""
You are a senior equity analyst and portfolio manager. Your expertise is in building highly accurate, thematic investment portfolios. This list is very noisy. Your job is to filter it aggressively.

### Investment Theme
{theme}

### Classification Rules
- **core**: The company's PRIMARY revenue driver or IP is this theme.
- **secondary**: A key supplier, enabler, or major beneficiary.
- **not_relevant**: A user of the technology or an unrelated business.

### Strict Instructions
1. Be skeptical. Do not be generous.
2. Use ONLY the provided business description.
3. If the description is vague or "N/A", classify as not_relevant.
4. If a company only USES the technology internally, it is not_relevant.

### Few-Shot Examples
Theme: AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software

Input:
{{"ticker": "NVDA", "company_name": "NVIDIA",
"business_description": "NVIDIA designs GPUs and AI hardware used for training and deploying AI models."}}

Output:
{{"ticker": "NVDA", "classification": "core",
"reasoning": "Company designs the core hardware that directly powers AI infrastructure."}}

Input:
{{"ticker": "WDAY", "company_name": "Workday",
"business_description": "Workday provides enterprise HR and finance SaaS with embedded AI features."}}

Output:
{{"ticker": "WDAY", "classification": "secondary",
"reasoning": "AI supports Workday’s HR SaaS platform but is not its primary business."}}

Input:
{{"ticker": "AON", "company_name": "Aon",
"business_description": "Aon provides insurance brokerage and risk management services."}}

Output:
{{"ticker": "AON", "classification": "not_relevant",
"reasoning": "Aon is an insurance and risk services firm, not an AI infrastructure provider."}}

### Output Format
Return a single JSON object with:
- ticker
- classification
- reasoning

### Analyze the following company to determine if it fits this Investment Theme: {theme}
---
company_name: {row.name}
ticker: {row.ticker}
business_description:
{row.prod_text}
---
"""
                    }
                ]
            )

            determine_theme = json.loads(response["message"]["content"])
            determine_theme["theme"] = theme
            determine_theme["ticker"] = row.ticker  # override in case model gets it wrong

            print(determine_theme)
            thematic_results.append(determine_theme)

        except Exception as e:
            print(e)
            continue


<All keys matched successfully>


{'ticker': 'COIN', 'classification': 'core', 'reasoning': "The company's primary revenue driver and core IP is its cryptocurrency exchange and digital asset custody services, as evidenced by its platform (Onramp) and its role as a major node operator on the Ethereum network. It is heavily invested in blockchain technology and infrastructure for scalability and automation.", 'theme': 'Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure'}
{'ticker': 'CRCL', 'classification': 'core', 'reasoning': "Circle's revenue model is built around its stablecoin network (USDC) and expanding into blockchain ecosystem services like tokenized funds, liquidity services, payments, and asset tokenization. This positions them directly within the blockchain ecosystem and digital asset space.", 'theme': 'Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure'}
{'ticker': 'MSTR', 'classification': 'core', 'reasoning': 'The company offers unique economic exposure to Bitcoin through eq

<All keys matched successfully>


{'ticker': 'ALAB', 'classification': 'core', 'reasoning': "The company's core focus is high-speed connectivity, directly benefiting from the increasing demand for specialized chips powering AI applications and data center growth. This aligns with the quantum computing, next-gen computing, and advanced semiconductors theme.", 'theme': 'Quantum Computing, Next-Gen Computing & Advanced Semiconductors'}
{'ticker': 'LSCC', 'classification': 'core', 'reasoning': 'Lattice is the last significant independent FPGA manufacturer, positioning it to capitalize on customer demand for diverse FPGA suppliers, a key area within next-gen computing and advanced semiconductors.', 'theme': 'Quantum Computing, Next-Gen Computing & Advanced Semiconductors'}
{'ticker': 'MRVL', 'classification': 'core', 'reasoning': "The company's strategic shift to custom silicon for data processing and storage, coupled with its expertise in networking and storage products, positions it as a core player in the next-gen comput

<All keys matched successfully>


...

{'ticker': 'TEAM', 'classification': 'core', 'reasoning': 'The company strategically integrates AI across its products, positioning it as a core growth driver and a key differentiator. This aligns with the theme of AI infrastructure, generative AI platforms, and enterprise software.', 'theme': 'AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software'}
{'ticker': 'NVDA', 'classification': 'core', 'reasoning': "NVI DIA's core revenue driver and IP is its GPUs, which are heavily used for AI training and deployment. The company's market position and strategic investments directly align with the AI infrastructure theme.", 'theme': 'AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software'}
{'ticker': 'PEGA', 'classification': 'core', 'reasoning': 'Pega is heavily investing in Generative AI (Pega GeNAI), which is a key growth driver and is integrated into their core platform. This aligns with the theme of AI platforms and enterprise software.', 'theme': 'AI Inf

<All keys matched successfully>


{'ticker': 'NET', 'classification': 'core', 'reasoning': "The company's business description explicitly mentions 'AI-Driven Infrastructure,' 'Edge Computing,' and 'Cloud Computing' as key areas of focus, indicating that these are its primary revenue drivers and core IP. The acquisitions in network security and cloud infrastructure further support this classification.", 'theme': 'Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure'}
{'ticker': 'EQIX', 'classification': 'core', 'reasoning': "Equinix's primary revenue driver and IP is its data center infrastructure, including investments in expanding its footprint, advanced infrastructure management tools, and partnerships to meet AI demand. The company's strategy is heavily focused on data centers and cloud computing.", 'theme': 'Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure'}
{'ticker': 'DLR', 'classification': 'core', 'reasoning': 'Digital Realty is a data center provider with a strong history of

<All keys matched successfully>


{'ticker': 'BEPC', 'classification': 'core', 'reasoning': 'The company operates approximately 13,948 MW of installed renewable energy capacity, directly related to renewable energy infrastructure. Being a subsidiary of Brookfield Renewable Partners further strengthens its position in the renewable energy sector.', 'theme': 'Nuclear, Renewable Energy, Grid Storage & Power for Digital Infrastructure'}
{'ticker': 'CEG', 'classification': 'core', 'reasoning': 'The company has significant investments in renewable energy (solar, wind) and a major contract to restart and upgrade a nuclear reactor, demonstrating a strong commitment to clean energy and maintaining a robust nuclear generation base. This aligns directly with the theme of Nuclear, Renewable Energy, Grid Storage & Power for Digital Infrastructure.', 'theme': 'Nuclear, Renewable Energy, Grid Storage & Power for Digital Infrastructure'}
{'ticker': 'AES', 'classification': 'core', 'reasoning': "AES's strategic shift, substantial renew

In [ ]:
thematic_results

[{'ticker': 'COIN',
  'classification': 'core',
  'reasoning': "The company's primary revenue driver and core IP is its cryptocurrency exchange and digital asset custody services, as evidenced by its platform (Onramp) and its role as a major node operator on the Ethereum network. It is heavily invested in blockchain technology and infrastructure for scalability and automation.",
  'theme': 'Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure'},
 {'ticker': 'CRCL',
  'classification': 'core',
  'reasoning': "Circle's revenue model is built around its stablecoin network (USDC) and expanding into blockchain ecosystem services like tokenized funds, liquidity services, payments, and asset tokenization. This positions them directly within the blockchain ecosystem and digital asset space.",
  'theme': 'Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure'},
 {'ticker': 'MSTR',
  'classification': 'core',
  'reasoning': 'The company offers unique economic exposure 

In [ ]:
thematic_results_df = pd.DataFrame(thematic_results)
thematic_results_df

,ticker,classification,reasoning,theme
0,COIN,core,The company's primary revenue driver and core ...,"Cryptocurrency, Digital Assets, Mining & Block..."
1,CRCL,core,Circle's revenue model is built around its sta...,"Cryptocurrency, Digital Assets, Mining & Block..."
2,MSTR,core,The company offers unique economic exposure to...,"Cryptocurrency, Digital Assets, Mining & Block..."
3,XYZ,core,The company is aggressively investing in its B...,"Cryptocurrency, Digital Assets, Mining & Block..."
4,DLR,not_relevant,Digital Realty is a data center provider focus...,"Cryptocurrency, Digital Assets, Mining & Block..."
5,CME,secondary,CME Group is a market infrastructure provider ...,"Cryptocurrency, Digital Assets, Mining & Block..."
6,AKAM,not_relevant,Akamai is a cloud infrastructure provider focu...,"Cryptocurrency, Digital Assets, Mining & Block..."
7,DJT,core,The company is planning to create a $2.5 billi...,"Cryptocurrency, Digital Assets, Mining & Block..."
8,FCX,not_relevant,"FCX is a copper mining company, which is a com...","Cryptocurrency, Digital Assets, Mining & Block..."
9,BLK,core,BlackRock is aggressively expanding its presen...,"Cryptocurrency, Digital Assets, Mining & Block..."


In [ ]:
thematic_results_df_corrected = thematic_results_df.copy()


# SME Review
There were no tickers that needed correction.
## Dealing with Misclassifications

## 1. Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure

In [ ]:
target_theme = "Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure"
to_secondary_1 = [
    "BLK", "CME", "DJT", "CBOE", "HOOD", "MSTR"
]
to_not_relevant_1 = [
    "ALAB", "NET", "DT",
    "MP", "AU", "FCX", "SCCO",
    "DLR", "MTZ", "BAM",
    "AKAM", "CRWD", "AXON",
    "GLOB", "EEFT", "NCNO",
    "BSY", "RGLD", "TTEK",
    "CIVI", "S"
]


# Core → Secondary
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_1) &
    (thematic_results_df_corrected["theme"] == target_theme) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

# Core → Not Relevant
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_1) &
    (thematic_results_df_corrected["theme"] == target_theme) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_1 + to_not_relevant_1) &
    (thematic_results_df_corrected["theme"] == target_theme)
]

,ticker,classification,reasoning,theme
2,MSTR,secondary,The company offers unique economic exposure to...,"Cryptocurrency, Digital Assets, Mining & Block..."
4,DLR,not_relevant,Digital Realty is a data center provider focus...,"Cryptocurrency, Digital Assets, Mining & Block..."
5,CME,secondary,CME Group is a market infrastructure provider ...,"Cryptocurrency, Digital Assets, Mining & Block..."
6,AKAM,not_relevant,Akamai is a cloud infrastructure provider focu...,"Cryptocurrency, Digital Assets, Mining & Block..."
7,DJT,secondary,The company is planning to create a $2.5 billi...,"Cryptocurrency, Digital Assets, Mining & Block..."
8,FCX,not_relevant,"FCX is a copper mining company, which is a com...","Cryptocurrency, Digital Assets, Mining & Block..."
9,BLK,secondary,BlackRock is aggressively expanding its presen...,"Cryptocurrency, Digital Assets, Mining & Block..."
10,AU,not_relevant,"AngloGold Ashanti is a gold mining company, wh...","Cryptocurrency, Digital Assets, Mining & Block..."
11,ALAB,not_relevant,The company's core focus is high-speed connect...,"Cryptocurrency, Digital Assets, Mining & Block..."
12,BSY,not_relevant,While Bentley Systems has a history of M&A in ...,"Cryptocurrency, Digital Assets, Mining & Block..."


## 2. Quantum Computing, Next-Gen Computing & Advanced Semiconductors

In [ ]:
target_theme = "Quantum Computing, Next-Gen Computing & Advanced Semiconductors"

to_secondary_2 = ["Q", "MTSI", "ON", "QRVO", "MCHP", "COHR", "NVDA"]
to_not_relevant_2 = ["QS"]

# Core → Secondary
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_2) &
    (thematic_results_df_corrected["theme"] == target_theme) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

# Core → Not Relevant
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_2) &
    (thematic_results_df_corrected["theme"] == target_theme) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"

## 3. Robotics, Automation, Autonomous Vehicles & Industrial AI Systems

In [ ]:
target_theme_3 = "Robotics, Automation, Autonomous Vehicles & Industrial AI Systems"

to_not_relevant_3 = ["TEAM"]
to_secondary_3 = ["DT", "CRUS", "CDNS", "SNPS", "MSI", "ALAB"]

# Core → Not Relevant
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_3) &
    (thematic_results_df_corrected["theme"] == target_theme_3) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"

# Core → Secondary
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_3) &
    (thematic_results_df_corrected["theme"] == target_theme_3) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_3 + to_not_relevant_3) &
    (thematic_results_df_corrected["theme"] == target_theme_3)
]

,ticker,classification,reasoning,theme
70,TEAM,not_relevant,The company strategically integrates AI across...,"Robotics, Automation, Autonomous Vehicles & In..."
73,DT,secondary,The company is increasingly focused on leverag...,"Robotics, Automation, Autonomous Vehicles & In..."
75,CRUS,secondary,The company has a significant focus on robotic...,"Robotics, Automation, Autonomous Vehicles & In..."
78,CDNS,secondary,The company's strategy heavily focuses on digi...,"Robotics, Automation, Autonomous Vehicles & In..."
83,SNPS,secondary,"The company is heavily investing in AI, partic...","Robotics, Automation, Autonomous Vehicles & In..."
86,MSI,secondary,The company's aggressive M&A strategy and focu...,"Robotics, Automation, Autonomous Vehicles & In..."
88,ALAB,secondary,The company's core focus is high-speed connect...,"Robotics, Automation, Autonomous Vehicles & In..."


## 4. AI-Powered Cybersecurity & Network Security Platforms

In [ ]:
target_theme_4 = "AI-Powered Cybersecurity & Network Security Platforms"

to_secondary_4 = [
    "PSN", "RNG", "TEAM", "SNPS", "CTSH", "PEGA",
    "AXON", "DT", "MSI", "DOX", "ZM", "ALAB", "PATH", "HUBS"
]
to_not_relevant_4 = ["AKAM"]

# Core → Secondary
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_4) &
    (thematic_results_df_corrected["theme"] == target_theme_4) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

# Core → Not Relevant
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_4) &
    (thematic_results_df_corrected["theme"] == target_theme_4) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_4 + to_not_relevant_4) &
    (thematic_results_df_corrected["theme"] == target_theme_4)
]

,ticker,classification,reasoning,theme
94,AXON,secondary,The company is strategically expanding into AI...,AI-Powered Cybersecurity & Network Security Pl...
96,DT,secondary,The company is increasingly focused on leverag...,AI-Powered Cybersecurity & Network Security Pl...
97,MSI,secondary,The company's aggressive M&A strategy and rece...,AI-Powered Cybersecurity & Network Security Pl...
101,AKAM,not_relevant,While Akamai provides content delivery and sec...,AI-Powered Cybersecurity & Network Security Pl...
104,RNG,secondary,The company is actively developing and deployi...,AI-Powered Cybersecurity & Network Security Pl...
106,DOX,secondary,Amdocs is investing in cloud computing and AI ...,AI-Powered Cybersecurity & Network Security Pl...
107,ZM,secondary,The company has made a strategic shift to an A...,AI-Powered Cybersecurity & Network Security Pl...
108,ALAB,secondary,The company's core focus is on high-speed conn...,AI-Powered Cybersecurity & Network Security Pl...
110,TEAM,secondary,The company strategically integrates AI and Ma...,AI-Powered Cybersecurity & Network Security Pl...
112,PATH,secondary,While UiPath is integrating AI and NLP capabil...,AI-Powered Cybersecurity & Network Security Pl...


## 5. Digital Finance, Payments, Neobanks & Fintech Infrastructure

In [ ]:
target_theme_5 = "Digital Finance, Payments, Neobanks & Fintech Infrastructure"

to_secondary_5 = ["COF", "JPM", "COLB", "AXS", "BR", "CRCL"]
to_not_relevant_5 = ["DLR", "AGO", "EFX", "SYF", "XP", "C"]

# Core → Secondary
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_5) &
    (thematic_results_df_corrected["theme"] == target_theme_5) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

# Core → Not Relevant
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_5) &
    (thematic_results_df_corrected["theme"] == target_theme_5) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_5 + to_not_relevant_5) &
    (thematic_results_df_corrected["theme"] == target_theme_5)
]

,ticker,classification,reasoning,theme
126,COLB,secondary,The company's digital banking solutions are a ...,"Digital Finance, Payments, Neobanks & Fintech ..."
127,AXS,secondary,The company's core business is built on provid...,"Digital Finance, Payments, Neobanks & Fintech ..."
132,CRCL,secondary,Circle's revenue model is built around its sta...,"Digital Finance, Payments, Neobanks & Fintech ..."
135,DLR,not_relevant,"Digital Realty is a data center provider, whic...","Digital Finance, Payments, Neobanks & Fintech ..."
136,BR,secondary,The company's primary revenue driver and IP is...,"Digital Finance, Payments, Neobanks & Fintech ..."
137,XP,not_relevant,While XP has a digital transformation in finan...,"Digital Finance, Payments, Neobanks & Fintech ..."
138,C,not_relevant,While Citi is undertaking technology transform...,"Digital Finance, Payments, Neobanks & Fintech ..."
139,EFX,not_relevant,While Equifax has a history of data breaches a...,"Digital Finance, Payments, Neobanks & Fintech ..."
141,COF,secondary,The business description highlights strategic ...,"Digital Finance, Payments, Neobanks & Fintech ..."
143,AGO,not_relevant,Assured Guaranty is a financial guaranty insur...,"Digital Finance, Payments, Neobanks & Fintech ..."


## 6. Metaverse, AR/VR, Gaming & Digital Reality Platforms

In [ ]:
target_theme_6 = "Metaverse, AR/VR, Gaming & Digital Reality Platforms"

to_secondary_6 = ["NVDA", "MSFT", "SMCI", "CDNS", "LITE", "PTC", "TEAM"]
to_not_relevant_6 = ["DLB", "ZM", "RNG", "HAS", "MAT"]

# Core → Secondary
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_6) &
    (thematic_results_df_corrected["theme"] == target_theme_6) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

# Core → Not Relevant
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_6) &
    (thematic_results_df_corrected["theme"] == target_theme_6) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"



In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_6 + to_not_relevant_6) &
    (thematic_results_df_corrected["theme"] == target_theme_6)
]

,ticker,classification,reasoning,theme
152,HAS,not_relevant,The company is strategically pivoting towards ...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
156,DLB,secondary,While Dolby is expanding into new markets like...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
157,NVDA,secondary,The company's primary revenue driver and IP is...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
158,ZM,not_relevant,The company has strategically pivoted to an AI...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
161,RNG,not_relevant,The company is heavily focused on AI-driven in...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
165,PTC,secondary,PTC is strategically focused on industrial IoT...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
166,LITE,secondary,The company's products are essential for high-...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
168,TEAM,secondary,While Atlassian is integrating AI into its pro...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
170,MAT,not_relevant,The company is strategically expanding into di...,"Metaverse, AR/VR, Gaming & Digital Reality Pla..."
174,MSFT,secondary,"Microsoft is heavily investing in AI, which is...","Metaverse, AR/VR, Gaming & Digital Reality Pla..."


## 7. Gene Editing, Synthetic Biology, AI Drug Discovery & Digital Health

In [ ]:
target_theme_7 = "Gene Editing, Synthetic Biology, AI Drug Discovery & Digital Health"

to_not_relevant_7 = ["BIIB", "AMGN", "RVMD", "INSM", "LLY", "APLS", "VKTX", "REGN"]

to_secondary_7 = ["EXAS", "RGEN", "HALO", "VRTX", "TECH", "A", "TMO", "RVTY"]


thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_7) &
    (thematic_results_df_corrected["theme"] == target_theme_7) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_7) &
    (thematic_results_df_corrected["theme"] == target_theme_7) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_7 + to_not_relevant_7) &
    (thematic_results_df_corrected["theme"] == target_theme_7)
]

,ticker,classification,reasoning,theme
183,BIIB,not_relevant,The company's primary revenue driver is drug d...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
188,EXAS,secondary,The company's strategy heavily focuses on liqu...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
190,RGEN,secondary,The company's primary revenue driver and IP is...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
192,HALO,secondary,Halozyme's core strategy is centered on its dr...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
193,AMGN,not_relevant,The company's strategic acquisitions and colla...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
194,RVMD,not_relevant,RVMD's core strategy and pipeline focus on RAS...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
195,INSM,not_relevant,Insmed's strategy explicitly leverages gene th...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
198,LLY,not_relevant,The company is actively leveraging strategic p...,"Gene Editing, Synthetic Biology, AI Drug Disco..."
199,VRTX,secondary,"Vertex's strategic portfolio expansion, includ...","Gene Editing, Synthetic Biology, AI Drug Disco..."
200,TECH,secondary,"The company's focus on diagnostics, exosomes, ...","Gene Editing, Synthetic Biology, AI Drug Disco..."


## 8. AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software

In [ ]:
target_theme_8 = "AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software"

to_secondary_8 = ["ALAB", "TEAM", "PEGA", "DOX", "DT", "RNG", "ZM", "SNOW", "G", "CTSH"]
to_not_relevant_8 = ["AKAM"]

thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_8) &
    (thematic_results_df_corrected["theme"] == target_theme_8) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_8) &
    (thematic_results_df_corrected["theme"] == target_theme_8) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"




In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_8 + to_not_relevant_8) &
    (thematic_results_df_corrected["theme"] == target_theme_8)
]

,ticker,classification,reasoning,theme
210,TEAM,secondary,The company strategically integrates AI across...,"AI Infrastructure, Chips, Generative AI Platfo..."
212,PEGA,secondary,Pega is heavily investing in Generative AI (Pe...,"AI Infrastructure, Chips, Generative AI Platfo..."
213,DT,secondary,The company is increasingly focused on leverag...,"AI Infrastructure, Chips, Generative AI Platfo..."
214,DOX,secondary,Amdocs is investing in cloud computing and AI ...,"AI Infrastructure, Chips, Generative AI Platfo..."
215,ALAB,secondary,Company's core focus is high-speed connectivit...,"AI Infrastructure, Chips, Generative AI Platfo..."
216,RNG,secondary,The company is actively developing and deployi...,"AI Infrastructure, Chips, Generative AI Platfo..."
221,ZM,secondary,Zoom has strategically pivoted to an AI-first ...,"AI Infrastructure, Chips, Generative AI Platfo..."
225,SNOW,secondary,Snowflake's Data Cloud platform is a key growt...,"AI Infrastructure, Chips, Generative AI Platfo..."
228,G,secondary,Genpact Cora is an AI-based platform designed ...,"AI Infrastructure, Chips, Generative AI Platfo..."
236,CTSH,secondary,The company's strategic focus is on AI-led aut...,"AI Infrastructure, Chips, Generative AI Platfo..."


## 9. Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure

In [ ]:
target_theme_9 = "Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure"

to_secondary_9 = ["ALAB", "SNOW", "NOW", "DDOG", "DT", "CFLT", "TWLO", "RNG", "CRWD", "S", "DBX", "TDC"]
to_not_relevant_9 = ["MANH"]

thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_9) &
    (thematic_results_df_corrected["theme"] == target_theme_9) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"

thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_not_relevant_9) &
    (thematic_results_df_corrected["theme"] == target_theme_9) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "not_relevant"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_9 + to_not_relevant_9) &
    (thematic_results_df_corrected["theme"] == target_theme_9)
]

,ticker,classification,reasoning,theme
248,ALAB,secondary,The company's core focus is high-speed connect...,"Cloud Computing, Data Centers, Hyperscalers & ..."
250,SNOW,secondary,Snowflake's Data Cloud platform is a key growt...,"Cloud Computing, Data Centers, Hyperscalers & ..."
251,NOW,secondary,"ServiceNow's subscription revenue growth, stra...","Cloud Computing, Data Centers, Hyperscalers & ..."
254,RNG,secondary,The company's business is centered around clou...,"Cloud Computing, Data Centers, Hyperscalers & ..."
255,CFLT,secondary,Confluent's core offering is a data streaming ...,"Cloud Computing, Data Centers, Hyperscalers & ..."
256,DDOG,secondary,DataDog's core offering is an observability pl...,"Cloud Computing, Data Centers, Hyperscalers & ..."
257,MANH,not_relevant,The company's primary revenue driver is its cl...,"Cloud Computing, Data Centers, Hyperscalers & ..."
259,TWLO,secondary,Twilio's core business is cloud communications...,"Cloud Computing, Data Centers, Hyperscalers & ..."
261,DT,secondary,DynaTrace is a observability platform focused ...,"Cloud Computing, Data Centers, Hyperscalers & ..."
262,CRWD,secondary,The company's business description highlights ...,"Cloud Computing, Data Centers, Hyperscalers & ..."


## 10. Nuclear, Renewable Energy, Grid Storage & Power for Digital Infrastructure

In [ ]:
target_theme_10 = "Nuclear, Renewable Energy, Grid Storage & Power for Digital Infrastructure"

to_secondary_10 = ["PWR", "MTZ", "MPWR", "BAM", "ETR"]

thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_10) &
    (thematic_results_df_corrected["theme"] == target_theme_10) &
    (thematic_results_df_corrected["classification"] == "core"),
    "classification"
] = "secondary"


In [ ]:
# Verify changes
thematic_results_df_corrected.loc[
    thematic_results_df_corrected["ticker"].isin(to_secondary_10) &
    (thematic_results_df_corrected["theme"] == target_theme_10)
]

,ticker,classification,reasoning,theme
284,PWR,secondary,The company's strategic acquisitions and proje...,"Nuclear, Renewable Energy, Grid Storage & Powe..."
285,MTZ,secondary,The company has a strategic focus on renewable...,"Nuclear, Renewable Energy, Grid Storage & Powe..."
287,MPWR,secondary,The company's core product offerings – energy-...,"Nuclear, Renewable Energy, Grid Storage & Powe..."
298,BAM,secondary,BAM's investment focus is explicitly on renewa...,"Nuclear, Renewable Energy, Grid Storage & Powe..."
299,ETR,secondary,ETR has a significant generating capacity (nuc...,"Nuclear, Renewable Energy, Grid Storage & Powe..."


## Handling overly broad classifications

In [ ]:
# Checked for tickers mappes to multiple investment themes
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

thematic_results_df_corrected[
    thematic_results_df_corrected.duplicated(subset="ticker", keep=False)
].sort_values("ticker")

,ticker,classification,reasoning,theme
6,AKAM,not_relevant,Akamai is a cloud infrastructure provider focu...,"Cryptocurrency, Digital Assets, Mining & Block..."
243,AKAM,core,The company's primary revenue driver is cloud ...,"Cloud Computing, Data Centers, Hyperscalers & ..."
237,AKAM,not_relevant,While Akamai provides cloud infrastructure ser...,"AI Infrastructure, Chips, Generative AI Platfo..."
101,AKAM,not_relevant,While Akamai provides content delivery and sec...,AI-Powered Cybersecurity & Network Security Pl...
215,ALAB,secondary,Company's core focus is high-speed connectivit...,"AI Infrastructure, Chips, Generative AI Platfo..."
248,ALAB,secondary,The company's core focus is high-speed connect...,"Cloud Computing, Data Centers, Hyperscalers & ..."
11,ALAB,not_relevant,The company's core focus is high-speed connect...,"Cryptocurrency, Digital Assets, Mining & Block..."
88,ALAB,secondary,The company's core focus is high-speed connect...,"Robotics, Automation, Autonomous Vehicles & In..."
30,ALAB,core,The company's core focus is high-speed connect...,"Quantum Computing, Next-Gen Computing & Advanc..."
108,ALAB,secondary,The company's core focus is on high-speed conn...,AI-Powered Cybersecurity & Network Security Pl...


In [ ]:
# Filtered tickers in multiple investment themes classifed as 'core'
thematic_results_df_corrected[
    (thematic_results_df_corrected.duplicated(subset="ticker", keep=False)) &
    (thematic_results_df_corrected["classification"] == "core")
].sort_values("ticker")

,ticker,classification,reasoning,theme
243,AKAM,core,The company's primary revenue driver is cloud ...,"Cloud Computing, Data Centers, Hyperscalers & ..."
30,ALAB,core,The company's core focus is high-speed connect...,"Quantum Computing, Next-Gen Computing & Advanc..."
103,ANET,core,The company's primary revenue driver is AI-dri...,AI-Powered Cybersecurity & Network Security Pl...
245,ANET,core,The company's primary revenue driver and IP is...,"Cloud Computing, Data Centers, Hyperscalers & ..."
226,ANET,core,The company's primary revenue driver is AI-dri...,"AI Infrastructure, Chips, Generative AI Platfo..."
238,AVGO,core,Broadcom is actively shifting towards software...,"AI Infrastructure, Chips, Generative AI Platfo..."
41,AVGO,core,Broadcom is actively developing custom ASICs (...,"Quantum Computing, Next-Gen Computing & Advanc..."
232,CDNS,core,The company's strategy heavily focuses on digi...,"AI Infrastructure, Chips, Generative AI Platfo..."
50,CDNS,core,The company's strategy heavily focuses on digi...,"Quantum Computing, Next-Gen Computing & Advanc..."
230,CGNX,core,Strategic focus on deep learning and acquisiti...,"AI Infrastructure, Chips, Generative AI Platfo..."


In [ ]:
# Manually check and change classification from core to secondary using ids
to_secondary = [103, 226, 243, 30, 41, 230, 114, 224, 91, 244, 249, 227, 92, 72, 71, 102, 77, 222, 3, 233, 32]

# Update the classification column for all of them at once
thematic_results_df_corrected.loc[to_secondary, 'classification'] = 'secondary'

In [ ]:
core_thematic_df = thematic_results_df_corrected.loc[thematic_results_df_corrected.classification=='core']
core_thematic_df

,ticker,classification,reasoning,theme
0,COIN,core,The company's primary revenue driver and core ...,"Cryptocurrency, Digital Assets, Mining & Block..."
1,CRCL,core,Circle's revenue model is built around its sta...,"Cryptocurrency, Digital Assets, Mining & Block..."
31,LSCC,core,Lattice is the last significant independent FP...,"Quantum Computing, Next-Gen Computing & Advanc..."
34,QCOM,core,Qualcomm is strategically targeting the rapidl...,"Quantum Computing, Next-Gen Computing & Advanc..."
35,INTC,core,Significant investment in manufacturing capaci...,"Quantum Computing, Next-Gen Computing & Advanc..."
37,AMD,core,Strategic partnership with OpenAI to develop A...,"Quantum Computing, Next-Gen Computing & Advanc..."
39,AMAT,core,Applied Materials is the dominant player in se...,"Quantum Computing, Next-Gen Computing & Advanc..."
42,GFS,core,The company's primary revenue driver and IP is...,"Quantum Computing, Next-Gen Computing & Advanc..."
44,LRCX,core,Lam Research's core business is semiconductor ...,"Quantum Computing, Next-Gen Computing & Advanc..."
47,SNPS,core,The company's primary revenue driver and IP is...,"Quantum Computing, Next-Gen Computing & Advanc..."


In [ ]:
core_thematic_df.to_csv("final_thematic_baskets.csv", index=False)

In [ ]:
# display
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)
! pip3 install yfinance

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
theme2tickers = {}
theme2prices = {}

for gn, gdf in core_thematic_df.groupby('theme'):
    tickers = gdf['ticker'].astype(str).tolist()
    theme2tickers[gn] = tickers

    theme2prices[gn] = yf.download(
        tickers,
        period="5y"
    )

/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/3347311062.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  7 of 7 completed
/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/3347311062.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  9 of 9 completed
/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/3347311062.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  14 of 14 completed
/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/3347311062.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  2 of 2 completed
/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/334731106

In [ ]:
theme2tickers

{'AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software': ['NVDA',
  'PLTR',
  'SNPS',
  'CDNS',
  'GOOG',
  'AVGO',
  'INTC'],
 'AI-Powered Cybersecurity & Network Security Platforms': ['FTNT',
  'PANW',
  'GEN',
  'CRWD',
  'ZS',
  'S',
  'ESTC',
  'LDOS',
  'SAIL'],
 'Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure': ['NET',
  'EQIX',
  'DLR',
  'ANET',
  'SMCI',
  'NTNX',
  'HPE',
  'NTAP',
  'CSCO',
  'PSTG',
  'MSFT',
  'ORCL',
  'MRVL',
  'MDB'],
 'Cryptocurrency, Digital Assets, Mining & Blockchain Infrastructure': ['COIN',
  'CRCL'],
 'Digital Finance, Payments, Neobanks & Fintech Infrastructure': ['EEFT',
  'FIS',
  'NU',
  'SOFI',
  'NCNO',
  'V',
  'XYZ',
  'FOUR',
  'MA',
  'FI',
  'JKHY',
  'WU',
  'SSNC',
  'GPN',
  'PCTY',
  'BILL'],
 'Gene Editing, Synthetic Biology, AI Drug Discovery & Digital Health': ['CAI',
  'RARE',
  'IONS',
  'MRNA',
  'SRPT',
  'NTRA',
  'ILMN',
  'ALNY',
  'TEM',
  'PODD',
  'CERT',
  'UTHR'],
 'Metaverse

In [ ]:
theme2tickers ['QQQ'] = ['QQQ']
theme2prices ['QQQ'] = yf.download(['QQQ'], period='5y')
theme2tickers ['ARKK'] = ['ARKK' ]
theme2prices ['ARKK'] = yf.download(['ARKK'], period='5y')
leveraged_etfs = [
    "QQQ",
    "SOXX",
    "XLK",
    "META", # From FANG+ and FBL
    "AAPL", # From FANG+ and AAPU
    "AMZN", # From FANG+ and AMZU
    "NFLX", # From FANG+
    "MSFT", # From FANG+
    "GOOGL", # From FANG+ and GGLL
    "CRWD", # From FANG+ II
    "NVDA", # From FANG+ and NVDL, NVDU
    "NOW", # From FANG+
    "AVGO", # From FANG+ and AVGX
    "TSLA", # TSLL, TSLT, TSLR, TSLG, TSL
    "MSTR", # MSTU, MSTX
    "AMD", # AMDL
    "COIN", # CONL
    "PLTR", # PLTU, PTIR
    "SMCI", # SMCX, SMCL
    "HOOD",
    "PANW"
    ]

theme2tickers['leveraged_etfs'] = leveraged_etfs
theme2prices['leveraged_etfs'] = yf.download(leveraged_etfs, period ='5y')

/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/4230735970.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/4230735970.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/4230735970.py:30: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  21 of 21 completed


In [ ]:
theme20HLC = {}
for gn in theme2prices:
    prices = theme2prices[gn]

    returns = prices.Close.pct_change()

    vol = returns.std()
    weights = 1 / vol
    weights = weights / weights.sum()

    portfolio_returns = (returns * weights).sum(axis=1)

    C = (portfolio_returns + 1).cumprod()
    O=(theme2prices[gn].Open/theme2prices[gn].Close).mean(axis=1) * C
    H=(theme2prices[gn].High/theme2prices[gn].Close).mean(axis=1) * C
    L=(theme2prices [gn].Low/theme2prices[gn].Close).mean(axis=1) * C
    theme20HLC[gn] = [O.iloc[1:],H.iloc[1:],L.iloc[1:],C.iloc[1:]]

/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/4066878527.py:5: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/4066878527.py:5: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/var/folders/hh/x1554b3j0vd5xd5rjt1lp1jm0000gn/T/ipykernel_75961/4066878527.py:5: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

/var/folde

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

def create_thematic_candlestick(theme_name, ohlc_data, period='1M'):
    """
    Create a candlestick chart for a thematic portfolio.

    Args:
        theme_name: Name of the investment theme
        ohlc_data: List containing [Open, High, Low, Close] series
        period: Resampling period ('1D', '1W', '1M', '3M')
    """
    O, H, L, C = ohlc_data

    # Create a DataFrame for easier manipulation
    df = pd.DataFrame({
        'Open': O,
        'High': H,
        'Low': L,
        'Close': C
    })

    # Resample to desired period for cleaner candlesticks
    df_resampled = df.resample(period).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }).dropna()

    # Create candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=df_resampled.index,
        open=df_resampled['Open'],
        high=df_resampled['High'],
        low=df_resampled['Low'],
        close=df_resampled['Close'],
        name=theme_name,
        increasing_line_color='#26a69a',  # Green for up days
        decreasing_line_color='#ef5350'   # Red for down days
    )])

    # Calculate performance metrics
    total_return = (df_resampled['Close'].iloc[-1] / df_resampled['Close'].iloc[0] - 1) * 100
    annualized_return = (df_resampled['Close'].iloc[-1] / df_resampled['Close'].iloc[0]) ** (252 / len(df)) - 1

    # Update layout
    fig.update_layout(
        title={
            'text': f'{theme_name}<br><sub>Total Return: {total_return:.1f}% | Annualized: {annualized_return*100:.1f}%</sub>',
            'x': 0.5,
            'xanchor': 'center'
        },
        yaxis_title='Portfolio Value (Base = 1.0)',
        xaxis_title='Date',
        xaxis_rangeslider_visible=True,
        height=600,
        template='plotly_white',
        hovermode='x unified',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            ),
            type="date"
        )
    )

    return fig

def create_multi_theme_comparison(theme2OHLC, themes_to_compare=None):
    """
    Create a comparison chart showing multiple themes' performance.

    Args:
        theme2OHLC: Dictionary mapping theme names to OHLC data
        themes_to_compare: List of theme names to compare (None = all)
    """
    if themes_to_compare is None:
        themes_to_compare = list(theme2OHLC.keys())

    fig = go.Figure()

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
              '#1a5276', '#f39c12'] # Added more colors

    for idx, theme in enumerate(themes_to_compare):
        if theme not in theme2OHLC:
            continue

        # Use closing prices for line chart comparison
        close_prices = theme2OHLC[theme][3]  # Close is the 4th element

        fig.add_trace(go.Scatter(
            x=close_prices.index,
            y=close_prices.values,
            mode='lines',
            name=theme[:40] + '...' if len(theme) > 40 else theme, # Shorten long names
            line=dict(color=colors[idx % len(colors)], width=2),
            hovertemplate='%{y:.3f}<extra></extra>'
        ))

    fig.update_layout(
        title='Thematic Portfolio Performance Comparison',
        yaxis_title='Cumulative Return (Base = 1.0)',
        xaxis_title='Date',
        height=700,
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02
        ),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            ),
            type="date"
        )
    )

    return fig

def create_theme_correlation_heatmap(theme2OHLC):
    """Create a correlation heatmap between different themes."""

    # Create DataFrame with closing prices for all themes
    close_prices_df = pd.DataFrame()
    for theme, ohlc in theme2OHLC.items():
        theme_short = theme[:30] + '...' if len(theme) > 30 else theme
        close_prices_df[theme_short] = ohlc[3]  # Close prices

    # Calculate returns
    returns_df = close_prices_df.pct_change().dropna()

    # Calculate correlation
    correlation_matrix = returns_df.corr()

    fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        colorscale='RdBu',
        zmid=0,
        text=correlation_matrix.values.round(2),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(title="Correlation")
    ))

    fig.update_layout(
        title='Thematic Portfolio Correlation Matrix (Daily Returns)',
        height=800,
        width=1000,
        xaxis_tickangle=-45
    )
    return fig



In [ ]:
import plotly.io as pio
pio.renderers.default = "vscode"

In [ ]:
thematic_candlestick_vis = create_thematic_candlestick(
    'AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software Candlestick Diagram', theme20HLC['AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software'], period = '1D'
)
thematic_candlestick_vis.show()

In [ ]:
vis_multi_theme_comparison = create_multi_theme_comparison(theme20HLC)
vis_multi_theme_comparison.show()

In [ ]:
technology_themes =['Quantum Computing, Next-Gen Computing & Advanced Semiconductors',
'Robotics, Automation, Autonomous Vehicles & Industrial AI Systems','AI Infrastructure, Chips, Generative AI Platforms & Enterprise Software',
'Cloud Computing, Data Centers, Hyperscalers & Network Infrastructure',
]

vis_tech_themes = create_multi_theme_comparison(theme20HLC,themes_to_compare=technology_themes)
vis_tech_themes.show()

In [ ]:
vis_corr_heatmap = create_theme_correlation_heatmap(theme20HLC)
vis_corr_heatmap.show()

In [ ]:
def create_thematic_dashboard(theme2OHLC, top_n=4):
    # Calculate final returns to identify top performers
    theme_returns = {name: ohlc[3].iloc[-1] for name, ohlc in theme2OHLC.items()}
    top_themes = sorted(theme_returns.items(), key=lambda x: x[1], reverse=True)[:top_n]

    # Create the subplot grid (2x2)
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"{t[:25]}... ({r:.2f}x)" for t, r in top_themes],
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )

    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

    # Loop through top themes and add to grid
    for idx, ((theme_name, _), (row, col)) in enumerate(zip(top_themes, positions)):
        O, H, L, C = theme2OHLC[theme_name]

        # Add candlestick to specific subplot
        fig.add_trace(
            go.Candlestick(
                x=C.index, open=O, high=H, low=L, close=C,
                name=theme_name,
                increasing_line_color='#26a69a',
                decreasing_line_color='#ef5350',
                showlegend=False # Keeps the dashboard clean
            ),
            row=row, col=col
        )

    fig.update_layout(
        title_text="Top Performing Thematic Portfolios",
        height=800,
        template='plotly_white',
        xaxis_rangeslider_visible=False # Sliders take too much room in grids
    )

    return fig

# Execute
dashboard = create_thematic_dashboard(theme20HLC)
dashboard.show()